In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
root_path = "/content/gdrive/My Drive/Endoscopy Disease Challenge/"

In [ ]:
!pip install albumentations --upgrade
!pip install efficientnet-pytorch=="0.6.3"

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
import albumentations.pytorch as AT
from data import EndoscopyDataset
from nets import EfficientUNetPP
from losses import WeightedDiceLoss
from engines import train_fn

In [4]:
model_name, encoder = "unet", "efficientnet-b2"
image_size, n_classes = 512, 5
batch_size, num_workers = 8, 4
start_epoch, total_epochs = 1, 80
device, device_ids = torch.device("cuda:0"), [0]

In [ ]:
df = pd.read_csv(root_path + "data.csv")
folds = 5
print("Num of Folds: {}\n".format(folds))
for fold in range(1, 6):
    head = "fold {:2}/{:2}".format(fold, folds)
    print(head + "\n" + "-"*(len(head)))

    train_df = df[df["Fold"] != fold].reset_index(drop=True)
    train_transform = A.Compose([
        A.RandomResizedCrop(height=image_size, width=image_size, scale=(0.8, 1.0)), 
        A.ShiftScaleRotate(), 
        A.Flip(), 
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2), 
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20), 
        A.Normalize(), 
        AT.ToTensor()
    ])
    train_loader = torch.utils.data.DataLoader(
        EndoscopyDataset(
            df=train_df, 
            image_folder=root_path + "images", mask_folder=root_path + "masks1", 
            transform=train_transform
        ), batch_size=batch_size, num_workers=num_workers, shuffle=True
    )
    eval_df = df[df["Fold"] == fold].reset_index(drop=True)
    eval_transform = A.Compose([
        A.Resize(image_size, image_size), 
        A.Normalize(), 
        AT.ToTensor()
    ])
    eval_loader = torch.utils.data.DataLoader(
        EndoscopyDataset(
            df=eval_df, 
            image_folder=root_path + "images", mask_folder=root_path + "masks1", 
            transform=eval_transform
        ), batch_size=batch_size, num_workers=num_workers
    )

    loaders = {
        "train": train_loader, 
        "eval": eval_loader
    }

    model = EfficientUNetPP(encoder=encoder, encoder_trainable=True, n_classes=n_classes)
    criterion = WeightedDiceLoss()
    optimizer = optim.Adam(model.parameters(), lr=5e-4)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5)

    train_fn(
        loaders, model, criterion, optimizer, lr_scheduler, 
        start_epoch, total_epochs, device, device_ids, 
        save_path=root_path + "disease segmentation/ckps/{}_{}_fold{}.pt".format(model_name, encoder, fold)
    )